In [1]:
from shapely.geometry import MultiPolygon
import json
from shapely.geometry import shape, mapping
from shapely.ops import unary_union
import os
import csv

In [2]:
file = "..\\rawData\\countries.geoJson"

with open(file, 'r') as f:
    data = json.load(f)

In [3]:
len(data['features'])

258

## Correct missing ISO

In [10]:

# list all features name where 'ISO3166-1-Alpha-3' == -99
missingIso = [
    feature['properties']['name'] for feature in data['features']
    if feature['properties']['ISO3166-1-Alpha-3'] == '-99'
]
missingIso

missingIso = {
    "France": "FRA",
    "Somaliland": "SOM",
    "Kosovo": "KOS",
}

# replace -99 with corresponding name
for feature in data['features']:
    if feature['properties']['name'] in missingIso:
        feature['properties']['ISO3166-1-Alpha-3'] = missingIso[feature['properties']['name']]


In [11]:
from shapely.geometry import shape, MultiPolygon, Polygon, mapping
from shapely.ops import transform
from pyproj import Transformer
import os, csv, json

def createCoordsFile(format="csv", idx=0, tolerance=0.02, output_dir='../processedData', crs="EPSG:4326"):

    if crs == "EPSG:4326":
        crs_path = "epsg4326"
    elif crs == "EPSG:3857":
        crs_path = "epsg3857"

    feature = data['features'][idx]
    iso3 = feature['properties']['ISO3166-1-Alpha-3']
    geometry = shape(feature['geometry'])

    # Reproject geometry if needed
    if crs == "EPSG:3857":
        transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
        geometry = transform(transformer.transform, geometry)

    simplified_geom = geometry.simplify(tolerance, preserve_topology=True)

    # Ensure geometry is iterable
    if isinstance(simplified_geom, Polygon):
        polygons = [simplified_geom]
    elif isinstance(simplified_geom, MultiPolygon):
        polygons = list(simplified_geom.geoms)
    else:
        raise TypeError(f"Unsupported geometry type: {simplified_geom.geom_type}")

    # Create output folder
    tol_str = str(tolerance).replace('.', '_')
    output_dir = f'../processedData/{format}/{crs_path}/tol{tol_str}'
    os.makedirs(output_dir, exist_ok=True)

    if format == "csv":
        csv_file = os.path.join(output_dir, f'{iso3}.csv')
        with open(csv_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['poly_idx', 'line_idx', 'lat', 'lon'] if crs == "EPSG:4326" else ['poly_idx', 'line_idx', 'y', 'x'])

            for poly_idx, polygon in enumerate(polygons):
                outline = polygon.boundary

                if outline.geom_type == 'MultiLineString':
                    lines = [list(line.coords) for line in outline.geoms]
                else:
                    lines = [list(outline.coords)]

                for line_idx, line in enumerate(lines):
                    for coord in line:
                        if isinstance(coord, (list, tuple)) and len(coord) == 2:
                            x, y = coord
                            writer.writerow([poly_idx, line_idx, y, x])
                        else:
                            print(f"Warning: Invalid coordinate format: {coord}")

    elif format == "json":
        outline_coords = []
        for polygon in polygons:
            outline = polygon.boundary
            if outline.geom_type == 'MultiLineString':
                outline_coords.extend([list(line.coords) for line in outline.geoms])
            else:
                outline_coords.append(list(outline.coords))

        with open(os.path.join(output_dir, f'{iso3}.json'), 'w') as f:
            json.dump({"coordinates": outline_coords}, f, indent=2)

    print(f"Created {iso3} at {os.path.join(output_dir, f'{iso3}.{format}')}")


In [12]:
for i in range(len(data['features'])):
    createCoordsFile(format='json', idx=i, tolerance=0, output_dir='../processedData')
    createCoordsFile(format='json', crs="EPSG:3857", idx=i, tolerance=0, output_dir='../processedData')

Created IDN at ../processedData/json/epsg4326/tol0\IDN.json
Created IDN at ../processedData/json/epsg3857/tol0\IDN.json
Created MYS at ../processedData/json/epsg4326/tol0\MYS.json
Created MYS at ../processedData/json/epsg3857/tol0\MYS.json
Created CHL at ../processedData/json/epsg4326/tol0\CHL.json
Created CHL at ../processedData/json/epsg3857/tol0\CHL.json
Created BOL at ../processedData/json/epsg4326/tol0\BOL.json
Created BOL at ../processedData/json/epsg3857/tol0\BOL.json
Created PER at ../processedData/json/epsg4326/tol0\PER.json
Created PER at ../processedData/json/epsg3857/tol0\PER.json
Created ARG at ../processedData/json/epsg4326/tol0\ARG.json
Created ARG at ../processedData/json/epsg3857/tol0\ARG.json
Created -99 at ../processedData/json/epsg4326/tol0\-99.json
Created -99 at ../processedData/json/epsg3857/tol0\-99.json
Created CYP at ../processedData/json/epsg4326/tol0\CYP.json
Created CYP at ../processedData/json/epsg3857/tol0\CYP.json
Created IND at ../processedData/json/eps

In [ ]:
# createCoordsFile(format='csv', idx=90, tolerance=0, output_dir='../processedData')
